In [ ]:
Kaius Evans
Regression Analysis of Year and State Fixed effects on CO2 Emissions regressed on energy prices and US State GDP 
between 1997-2019.

In [ ]:
    The relationships between economic activity and carbon pollution are usually investigated through a reduced-form energy demand 
model that links energy consumption to energy prices and aggregate economic activity.
    
    The data comes from the U.S. Department of Energy, the U.S. Energy Information Administration and the Bureau of Economic Analysis.

In [ ]:
IMPORT AND CLEAN DATASETS

In [1]:
import pandas as pd
co2_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/co2_FINAL.xlsx"
co2_data = pd.read_excel(co2_path, skiprows=4, nrows=51)  
co2_data = co2_data.rename(columns={co2_data.columns[0]: "State"})

year_cols = [col for col in co2_data.columns if isinstance(col, int) and 1997 <= col <= 2019]

filtered_co2_data = co2_data[["State"] + year_cols]
filtered_co2_data = filtered_co2_data.rename(columns={year: f"co2.{year}" for year in year_cols})
filtered_co2_data = filtered_co2_data.reset_index().rename(columns={"index": "row_id"})

co2_long = pd.wide_to_long(filtered_co2_data, 
                          stubnames="co2", 
                          i=["State", "row_id"],
                          j="Year", 
                          sep=".")

co2_long = co2_long.reset_index()

co2_long = co2_long.drop("row_id", axis=1)
codx = co2_long.reset_index(drop=True)
print("CO2 Data (codx) - First 50 rows:")
print(codx.head(50))
print("\nDataFrame dtypes:")
print(codx.dtypes)

CO2 Data (codx) - First 50 rows:
        State  Year         co2
0      ALASKA  1997   41.217128
1      ALASKA  1998   42.236285
2      ALASKA  1999   42.864469
3      ALASKA  2000   43.575045
4      ALASKA  2001   42.548535
5      ALASKA  2002   42.736578
6      ALASKA  2003   42.890870
7      ALASKA  2004   46.193065
8      ALASKA  2005   47.485308
9      ALASKA  2006   45.417570
10     ALASKA  2007   43.653642
11     ALASKA  2008   39.079171
12     ALASKA  2009   37.283488
13     ALASKA  2010   37.137284
14     ALASKA  2011   37.085337
15     ALASKA  2012   36.155281
16     ALASKA  2013   34.049190
17     ALASKA  2014   33.919405
18     ALASKA  2015   35.027804
19     ALASKA  2016   33.406902
20     ALASKA  2017   33.726966
21     ALASKA  2018   34.514589
22     ALASKA  2019   34.259440
23   ALABAMA   1997  134.122751
24   ALABAMA   1998  133.525606
25   ALABAMA   1999  135.848198
26   ALABAMA   2000  142.302750
27   ALABAMA   2001  133.309972
28   ALABAMA   2002  138.301049
29   AL

In [2]:
coal_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/pr_ex_cl_es.xlsx"
coal_data = pd.read_excel(coal_path, sheet_name="Coal Prices", skiprows=2, nrows=51)  
coal_data = coal_data.rename(columns={coal_data.columns[0]: "State"})

year_cols = [col for col in coal_data.columns if isinstance(col, int) and 1997 <= col <= 2019]

filtered_coal_data = coal_data[["State"] + year_cols]
filtered_coal_data = filtered_coal_data.rename(columns={year: f"coal.{year}" for year in year_cols})
filtered_coal_data = filtered_coal_data.reset_index().rename(columns={"index": "row_id"})

coal_long = pd.wide_to_long(filtered_coal_data, 
                           stubnames="coal", 
                           i=["State", "row_id"],
                           j="Year", 
                           sep=".")

coal_long = coal_long.reset_index()
coal_long = coal_long.sort_values(by=["State", "Year"])
coal_long = coal_long.drop("row_id", axis=1)


coal_dx = coal_long.reset_index(drop=True)
print("Coal Data (coal_dx) - First 5 rows:")
print(coal_dx.head())
print("\nDataFrame dtypes:")
print(coal_dx.dtypes)

Coal Data (coal_dx) - First 5 rows:
  State  Year  coal
0    AK  1997  2.18
1    AK  1998  2.06
2    AK  1999  2.12
3    AK  2000  1.87
4    AK  2001  1.89

DataFrame dtypes:
State     object
Year       int64
coal     float64
dtype: object


In [3]:
elec_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/pr_ex_cl_es.xlsx"
elec_data = pd.read_excel(elec_path, sheet_name="Electricity Prices", skiprows=2, nrows=51)  
elec_data = elec_data.rename(columns={elec_data.columns[0]: "State"})

year_cols = [col for col in elec_data.columns if isinstance(col, int) and 1997 <= col <= 2019]

filtered_elec_data = elec_data[["State"] + year_cols]
filtered_elec_data = filtered_elec_data.rename(columns={year: f"elec.{year}" for year in year_cols})
filtered_elec_data = filtered_elec_data.reset_index().rename(columns={"index": "row_id"})

elec_long = pd.wide_to_long(filtered_elec_data, 
                           stubnames="elec", 
                           i=["State", "row_id"],
                           j="Year", 
                           sep=".")
elec_long = elec_long.reset_index()
elec_long = elec_long.sort_values(by=["State", "Year"])
elec_long = elec_long.drop("row_id", axis=1)

elec_dx = elec_long.reset_index(drop=True)
print("Electricity Data (elec_dx) - First 5 rows:")
print(elec_dx.head())
print("\nDataFrame dtypes:")
print(elec_dx.dtypes)

Electricity Data (elec_dx) - First 5 rows:
  State  Year   elec
0    AK  1997  29.57
1    AK  1998  29.29
2    AK  1999  28.71
3    AK  2000  29.60
4    AK  2001  30.96

DataFrame dtypes:
State     object
Year       int64
elec     float64
dtype: object


In [4]:
petro_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/pr_ex_pa_ng.xlsx"
petro_data = pd.read_excel(petro_path, sheet_name="Petroleum Prices", skiprows=2, nrows=51)  
petro_data = petro_data.rename(columns={petro_data.columns[0]: "State"})
year_cols = [col for col in petro_data.columns if isinstance(col, int) and 1997 <= col <= 2019]
filtered_petro_data = petro_data[["State"] + year_cols]
filtered_petro_data = filtered_petro_data.rename(columns={year: f"petro.{year}" for year in year_cols})
filtered_petro_data = filtered_petro_data.reset_index().rename(columns={"index": "row_id"})
petro_long = pd.wide_to_long(filtered_petro_data, 
                           stubnames="petro", 
                           i=["State", "row_id"],
                           j="Year", 
                           sep=".")
petro_long = petro_long.reset_index()
petro_long = petro_long.sort_values(by=["State", "Year"])
petro_long = petro_long.drop("row_id", axis=1)
petro_dx = petro_long.reset_index(drop=True)
print("Petroleum Data (petro_dx) - First 5 rows:")
print(petro_dx.head())
print("\nDataFrame dtypes:")
print(petro_dx.dtypes)

Petroleum Data (petro_dx) - First 5 rows:
  State  Year  petro
0    AK  1997   6.98
1    AK  1998   5.54
2    AK  1999   6.07
3    AK  2000   8.54
4    AK  2001   7.99

DataFrame dtypes:
State     object
Year       int64
petro    float64
dtype: object


In [5]:
natur_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/pr_ex_pa_ng.xlsx"
natur_data = pd.read_excel(natur_path, sheet_name="Natural Gas Prices", skiprows=2, nrows=51)  
natur_data = natur_data.rename(columns={natur_data.columns[0]: "State"})
year_cols = [col for col in natur_data.columns if isinstance(col, int) and 1997 <= col <= 2019]
filtered_natur_data = natur_data[["State"] + year_cols]
filtered_natur_data = filtered_natur_data.rename(columns={year: f"natur.{year}" for year in year_cols})
filtered_natur_data = filtered_natur_data.reset_index().rename(columns={"index": "row_id"})
natur_long = pd.wide_to_long(filtered_natur_data, 
                           stubnames="natur", 
                           i=["State", "row_id"],
                           j="Year", 
                           sep=".")
natur_long = natur_long.reset_index()
natur_long = natur_long.sort_values(by=["State", "Year"])
natur_long = natur_long.drop("row_id", axis=1)
natur_dx = natur_long.reset_index(drop=True)
print("naturleum Data (natur_dx) - First 5 rows:")
print(natur_dx.head())
print("\nDataFrame dtypes:")
print(natur_dx.dtypes)

naturleum Data (natur_dx) - First 5 rows:
  State  Year  natur
0    AK  1997   2.08
1    AK  1998   2.02
2    AK  1999   1.92
3    AK  2000   1.97
4    AK  2001   2.58

DataFrame dtypes:
State     object
Year       int64
natur    float64
dtype: object


In [6]:
pop_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/us_pop_gdp.xlsx"
pop_data = pd.read_excel(pop_path, sheet_name="Total Population", skiprows=2, nrows=51)  
pop_data = pop_data.rename(columns={pop_data.columns[0]: "State"})
year_cols = [col for col in pop_data.columns if isinstance(col, int) and 1997 <= col <= 2019]
filtered_pop_data = pop_data[["State"] + year_cols]
filtered_pop_data = filtered_pop_data.rename(columns={year: f"pop.{year}" for year in year_cols})
filtered_pop_data = filtered_pop_data.reset_index().rename(columns={"index": "row_id"})
pop_long = pd.wide_to_long(filtered_pop_data, 
                           stubnames="pop", 
                           i=["State", "row_id"],
                           j="Year", 
                           sep=".")
pop_long = pop_long.reset_index()
pop_long = pop_long.sort_values(by=["State", "Year"])
pop_long = pop_long.drop("row_id", axis=1)
pop_dx = pop_long.reset_index(drop=True)
print("popleum Data (pop_dx) - First 5 rows:")
print(pop_dx.head())
print("\nDataFrame dtypes:")
print(pop_dx.dtypes)

popleum Data (pop_dx) - First 5 rows:
  State  Year  pop
0    AK  1997  613
1    AK  1998  620
2    AK  1999  625
3    AK  2000  628
4    AK  2001  634

DataFrame dtypes:
State    object
Year      int64
pop       int64
dtype: object


In [7]:
gdp_path = "/Users/kaiusevans/Downloads/MIDTERM-2/STATE_DATA/us_pop_gdp.xlsx"
gdp_data = pd.read_excel(gdp_path, sheet_name="Real GDP", skiprows=2, nrows=51)  
gdp_data = gdp_data.rename(columns={gdp_data.columns[0]: "State"})
year_cols = [col for col in gdp_data.columns if isinstance(col, int) and 1997 <= col <= 2019]
filtered_gdp_data = gdp_data[["State"] + year_cols]
filtered_gdp_data = filtered_gdp_data.rename(columns={year: f"gdp.{year}" for year in year_cols})
filtered_gdp_data = filtered_gdp_data.reset_index().rename(columns={"index": "row_id"})
gdp_long = pd.wide_to_long(filtered_gdp_data, 
                           stubnames="gdp", 
                           i=["State", "row_id"],
                           j="Year", 
                           sep=".")
gdp_long = gdp_long.reset_index()
gdp_long = gdp_long.sort_values(by=["State", "Year"])
gdp_long = gdp_long.drop("row_id", axis=1)
gdp_dx = gdp_long.reset_index(drop=True)
print("gdpleum Data (gdp_dx) - First 5 rows:")
print(gdp_dx.head())
print("\nDataFrame dtypes:")
print(gdp_dx.dtypes)

gdpleum Data (gdp_dx) - First 5 rows:
  State  Year    gdp
0    AK  1997  42339
1    AK  1998  41238
2    AK  1999  40788
3    AK  2000  39567
4    AK  2001  41020

DataFrame dtypes:
State    object
Year      int64
gdp       int64
dtype: object


In [ ]:
MERGE DATA AND DEFINE NEW DATAFRAM (DF)

In [8]:
state_map = {
    
    'AK': 'ALASKA', 'AL': 'ALABAMA', 'AR': 'ARKANSAS', 'AZ': 'ARIZONA', 
    'CA': 'CALIFORNIA', 'CO': 'COLORADO', 'CT': 'CONNECTICUT', 'DC': 'DISTRICT OF COLUMBIA',
    'DE': 'DELAWARE', 'FL': 'FLORIDA', 'GA': 'GEORGIA', 'HI': 'HAWAII', 
    'IA': 'IOWA', 'ID': 'IDAHO', 'IL': 'ILLINOIS', 'IN': 'INDIANA',
    'KS': 'KANSAS', 'KY': 'KENTUCKY', 'LA': 'LOUISIANA', 'MA': 'MASSACHUSETTS',
    'MD': 'MARYLAND', 'ME': 'MAINE', 'MI': 'MICHIGAN', 'MN': 'MINNESOTA', 
    'MO': 'MISSOURI', 'MS': 'MISSISSIPPI', 'MT': 'MONTANA', 'NC': 'NORTH CAROLINA',
    'ND': 'NORTH DAKOTA', 'NE': 'NEBRASKA', 'NH': 'NEW HAMPSHIRE', 'NJ': 'NEW JERSEY',
    'NM': 'NEW MEXICO', 'NV': 'NEVADA', 'NY': 'NEW YORK', 'OH': 'OHIO',
    'OK': 'OKLAHOMA', 'OR': 'OREGON', 'PA': 'PENNSYLVANIA', 'RI': 'RHODE ISLAND',
    'SC': 'SOUTH CAROLINA', 'SD': 'SOUTH DAKOTA', 'TN': 'TENNESSEE', 'TX': 'TEXAS',
    'UT': 'UTAH', 'VA': 'VIRGINIA', 'VT': 'VERMONT', 'WA': 'WASHINGTON',
    'WI': 'WISCONSIN', 'WV': 'WEST VIRGINIA', 'WY': 'WYOMING'
}

df = coal_dx.copy()
df = pd.merge(df, elec_dx, on=["State", "Year"])
df = pd.merge(df, petro_dx, on=["State", "Year"])
df = pd.merge(df, natur_dx, on=["State", "Year"])
df = pd.merge(df, pop_dx, on=["State", "Year"])
df = pd.merge(df, gdp_dx, on=["State", "Year"])

df['State_full'] = df['State'].map(state_map)

codx_temp = codx.copy()
codx_temp['State'] = codx_temp['State'].str.strip()

df = pd.merge(df, codx_temp, left_on=["State_full", "Year"], right_on=["State", "Year"], how="left")

df = df.drop(['State_full', 'State_y'], axis=1)
df = df.rename(columns={'State_x': 'State'})

print(df.head(50))
print(df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nTotal number of states in merged data:", df['State'].nunique())
print("States in merged data:", sorted(df['State'].unique()))

   State  Year  coal   elec  petro  natur   pop     gdp         co2
0     AK  1997  2.18  29.57   6.98   2.08   613   42339   41.217128
1     AK  1998  2.06  29.29   5.54   2.02   620   41238   42.236285
2     AK  1999  2.12  28.71   6.07   1.92   625   40788   42.864469
3     AK  2000  1.87  29.60   8.54   1.97   628   39567   43.575045
4     AK  2001  1.89  30.96   7.99   2.58   634   41020   42.548535
5     AK  2002  1.94  30.76   7.32   2.65   642   42916   42.736578
6     AK  2003  2.00  30.86   8.40   3.02   648   42133   42.890870
7     AK  2004  1.97  32.29  11.09   3.34   659   43752   46.193065
8     AK  2005  2.01  34.43  14.49   3.97   667   45065   47.485308
9     AK  2006  2.13  37.69  16.92   4.67   675   48538   45.417570
10    AK  2007  2.34  38.96  18.09   5.76   680   51144   43.653642
11    AK  2008  3.01  43.21  25.31   6.50   687   50891   39.079171
12    AK  2009  3.61  44.29  17.55   7.42   699   55911   37.283488
13    AK  2010  3.43  43.29  20.81   6.41   714 

In [9]:
del codx
del coal_dx
del elec_dx
del petro_dx
del natur_dx
del pop_dx
del gdp_dx

In [10]:
df = df[~df['State'].isin(['AK', 'HI', 'DC'])]

print(df.shape)
print(df.head())

(1104, 9)
   State  Year  coal   elec  petro  natur   pop     gdp         co2
23    AL  1997  1.57  15.76   8.35   4.68  4368  144457  134.122751
24    AL  1998  1.59  16.45   7.36   4.24  4405  149613  133.525606
25    AL  1999  1.50  16.39   7.94   4.34  4430  155172  135.848198
26    AL  2000  1.44  16.60  10.19   5.32  4452  157698  142.302750
27    AL  2001  1.44  16.61   9.90   7.22  4468  157138  133.309972


In [11]:
df['coal'] = df['coal'].replace(0, 0.5)

In [12]:
df['gdppc'] = df['gdp'] / df['pop']
print(df['gdppc'].describe())

print(df.head(50))

count    1104.000000
mean       48.221549
std         9.436042
min        29.986676
25%        41.245871
50%        46.788561
75%        53.736236
max        78.095238
Name: gdppc, dtype: float64
   State  Year  coal   elec  petro  natur   pop     gdp         co2      gdppc
23    AL  1997  1.57  15.76   8.35   4.68  4368  144457  134.122751  33.071658
24    AL  1998  1.59  16.45   7.36   4.24  4405  149613  133.525606  33.964359
25    AL  1999  1.50  16.39   7.94   4.34  4430  155172  135.848198  35.027540
26    AL  2000  1.44  16.60  10.19   5.32  4452  157698  142.302750  35.421833
27    AL  2001  1.44  16.61   9.90   7.22  4468  157138  133.309972  35.169651
28    AL  2002  1.45  16.92   9.29   5.55  4480  161258  138.301049  35.995089
29    AL  2003  1.49  17.41  10.46   7.30  4503  165250  139.781270  36.697757
30    AL  2004  1.57  18.01  12.58   7.84  4531  175917  141.984129  38.825204
31    AL  2005  1.89  19.14  16.07  10.49  4570  181871  143.520721  39.796718
32    AL  2006

In [ ]:
REGRESSION

In [13]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

reg1_co2 = smf.ols(
    formula = "np.log(co2) ~ np.log(coal) + np.log(elec) + np.log(petro) + np.log(natur) + np.log(gdppc) + I(np.log(gdppc) ** 2)", 
    data = df,
    eval_env = 1
).fit(cov_type = "HC1")

print(reg1_co2.summary())

                            OLS Regression Results                            
Dep. Variable:            np.log(co2)   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     44.77
Date:                Mon, 24 Mar 2025   Prob (F-statistic):           3.97e-49
Time:                        02:53:11   Log-Likelihood:                -1413.0
No. Observations:                1104   AIC:                             2840.
Df Residuals:                    1097   BIC:                             2875.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -76.55

In [14]:
reg2_co2 = smf.ols(
    formula = "np.log(co2) ~ np.log(coal) + np.log(elec) + np.log(petro) + np.log(natur) + np.log(gdppc) + I(np.log(gdppc) ** 2) + C(State)", 
    data = df,
    eval_env = 1
).fit(cov_type = "HC1")

print(reg2_co2.summary())

                            OLS Regression Results                            
Dep. Variable:            np.log(co2)   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     6693.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        02:53:11   Log-Likelihood:                 1395.0
No. Observations:                1104   AIC:                            -2682.
Df Residuals:                    1050   BIC:                            -2412.
Df Model:                          53                                         
Covariance Type:                  HC1                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.24

In [15]:
reg3_co2 = smf.ols(
    formula = "np.log(co2) ~ np.log(coal) + np.log(elec) + np.log(petro) + np.log(natur) + np.log(gdppc) + I(np.log(gdppc) ** 2) + C(State) + C(Year)", 
    data = df,
    eval_env = 1
).fit(cov_type = "HC1")

print(reg3_co2.summary())

                            OLS Regression Results                            
Dep. Variable:            np.log(co2)   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     4971.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        02:53:12   Log-Likelihood:                 1521.8
No. Observations:                1104   AIC:                            -2892.
Df Residuals:                    1028   BIC:                            -2511.
Df Model:                          75                                         
Covariance Type:                  HC1                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 1.23

In [17]:
from stargazer.stargazer import Stargazer

table = Stargazer([reg1_co2, reg2_co2, reg3_co2])
table.significant_digits(3)

display(table)

In [ ]:
INTERPRETATION

In [ ]:
    The results follow Economic Theory except for the coal and natur regressor. Although all the regressors signs (+ or -) follow 
Economic Theory, the coal and natur regressors should have a negative relationship (instead of a positive one) with emissons. As prices 
increase, there will be less buyers, or entities would prefer to save there energy due to the the increase in price, thus co2 emissions 
should fall. This may also indicate there are potential hidden Time Fixed Effects.

In [ ]:
    Including log(gdppc)^2 is essential to the regression. While log(gdppc) covers the linear relationship between income and emissions,
it does not measure how that relationship might change as income increases. log(gdppc)^2 accounts for this by determing whether or 
not the effect of income of emissions changes past a certain level. For example As gdppc increases, governments might invest in cleaner 
energy and environmental protection, reducing emissions.

In [ ]:
    Based on the OLS results, the implied shape of the model is an inverted U. This is the case since log(gdppc) is postive, but log(gdppc)^2 is 
negative. In real-world terms, this means that income does in fact increase emissions, but only up to a certain point, after which, 
emissions start to decline. 
    Based on the p-value of log(gdppc)^2 (.087), at a 5% statistical significance level, we fail to reject the null hypothesis. 

In [18]:
hypothesis = 'np.log(coal) = 0, np.log(elec) = 0, np.log(petro) = 0, np.log(natur) = 0'
joint_test = reg2_co2.f_test(hypothesis)

print(joint_test)

<F test: F=134.84009056010763, p=5.443527997442159e-93, df_denom=1.05e+03, df_num=4>


In [ ]:
The joint F-test strongly rejects the null hypothesis at the 5% significance level. This means that the energy price variables are 
jointly statistically significant predictors of CO₂ emissions.